In [1]:
import pickle as pkl
from copy import deepcopy
import jsonlines
from utils.my_util import cluster_mentions, remove_speaker_prefix
import json

In [2]:
speaker_dict = {}
with open('data/raw_source/dialogue_en/all_coref_data_en_finalized.json', 'r') as f:
    temp = json.load(f)
    for line in temp:
        scene_id = line['scene_id']
        speakers = []
        for sent in line['sentences']:
            speakers.append(" ".join(sent[:sent.index(":")]))
        speaker_dict[scene_id] = speakers

split_dict = {"train":[], "dev":[], "test":[]}
with open('data/raw_source/dialogue_zh/dev_temp.pkl', 'rb') as f:
    temp = pkl.load(f)
    for line in temp:
        split_dict['dev'].append(line['scene_id'])
with open('data/raw_source/dialogue_zh/test_temp.pkl', 'rb') as f:
    temp = pkl.load(f)
    for line in temp:
        split_dict['test'].append(line['scene_id'])
with open('data/raw_source/dialogue_zh/train_temp.pkl', 'rb') as f:
    temp = pkl.load(f)
    for line in temp:
        split_dict['train'].append(line['scene_id'][:-1])

In [3]:
print(split_dict)

{'train': ['s01e01c00t', 's01e01c01t', 's01e01c03t', 's01e01c04t', 's01e01c05t', 's01e01c06t', 's01e01c07t', 's01e01c08t', 's01e01c09t', 's01e01c10t', 's01e03c00t', 's01e10c04t', 's01e03c01t', 's01e03c02t', 's01e03c03t', 's01e03c04t', 's01e03c05t', 's01e03c06t', 's01e03c07t', 's01e03c08t', 's01e03c09t', 's01e04c00t', 's01e04c01t', 's01e04c02t', 's01e04c03t', 's01e04c04t', 's01e04c05t', 's01e04c06t', 's01e04c07t', 's01e04c08t', 's01e04c09t', 's01e04c10t', 's01e04c11t', 's01e04c12t', 's01e05c00t', 's01e05c01t', 's01e05c02t', 's01e05c03t', 's01e05c04t', 's01e05c05t', 's01e05c06t', 's01e05c07t', 's01e07c01t', 's01e07c02t', 's01e07c03t', 's01e07c04t', 's01e07c05t', 's01e09c00t', 's01e09c01t', 's01e09c02t', 's01e09c03t', 's01e09c04t', 's01e09c05t', 's01e09c06t', 's01e07c00t', 's01e09c07t', 's01e10c00t', 's01e10c01t', 's01e10c02t', 's01e10c03t', 's01e10c05t', 's01e10c07t', 's01e11c00t', 's01e11c01t', 's01e11c02t', 's01e11c03t', 's01e11c04t', 's01e11c05t', 's01e11c06t', 's01e11c07t', 's01e11c0

In [4]:
print(len(split_dict['train'])+len(split_dict['dev'])+len(split_dict['test']))

1224


In [6]:
print(955+134+133)
print(18477+2578+1909)

1222
22964


In [10]:
data = {}
all_ids = []
with open('data/raw_source/dialogue_en/all_coref_data_en_finalized.json', 'r') as f:
# with open('data/raw_source/dialogue_zh/dev-test-batch1_zh.json', 'r') as f:
    reader = jsonlines.Reader(f)
    for bulk in reader:
        for idx, instance in enumerate(bulk):
            # if idx>=5:
            #     break

            scene_id = instance['scene_id']
            if scene_id == "":
                continue
            sentences = instance['sentences']

            # for sent in sentences:
            #     print(sent.index(":"), sent)
            # print(sentences)
            # sentences = [[token for token in "".join(sent)] for sent in instance['sentences']]
            annotations = instance['annotations']
            all_ids.append(scene_id)
            speakers = speaker_dict[scene_id]
            answers = []
            for item in annotations:
                query = (item['query']['sentenceIndex'], item['query']['startToken']-1-sentences[item['query']['sentenceIndex']].index(":"), item['query']['endToken']-1-sentences[item['query']['sentenceIndex']].index(":"))
                antecedents = item['antecedents']
                if antecedents in [['n', 'o', 't', 'P', 'r', 'e', 's', 'e', 'n', 't'], ['null_projection'], ['empty_subtitle']]:
                    answers.append([query, "notPresent"])
                else:
                    temp_answer = []
                    for antecedent in antecedents:
                        if isinstance(antecedent, dict):
                            temp_answer.append((antecedent['sentenceIndex'], antecedent['startToken']-1-sentences[antecedent['sentenceIndex']].index(":"), antecedent['endToken']-1-sentences[antecedent['sentenceIndex']].index(":")))
                        else:
                            temp_answer = " ".join(antecedents)
                    answers.append([query, temp_answer])
            new_sentences = []
            for sent in sentences:
                new_sentences.append(sent[sent.index(":")+1:])

            data[scene_id] = {
                "sentences": new_sentences,
                "answers": answers,
                "speakers": speakers,
                "scene_id": scene_id
            }

            # data.append({
            #     "sentences": new_sentences,
            #     "answers": answers,
            #     "speakers": speakers,
            #     "scene_id": scene_id
            # })

In [11]:
print(data.keys())

dict_keys(['s01e01c00t', 's01e01c01t', 's01e01c03t', 's01e01c04t', 's01e01c05t', 's01e01c06t', 's01e01c07t', 's01e01c08t', 's01e01c09t', 's01e01c10t', 's01e03c00t', 's01e10c04t', 's01e03c01t', 's01e03c02t', 's01e03c03t', 's01e03c04t', 's01e03c05t', 's01e03c06t', 's01e03c07t', 's01e03c08t', 's01e03c09t', 's01e04c00t', 's01e04c01t', 's01e04c02t', 's01e04c03t', 's01e04c04t', 's01e04c05t', 's01e04c06t', 's01e04c07t', 's01e04c08t', 's01e04c09t', 's01e04c10t', 's01e04c11t', 's01e04c12t', 's01e05c00t', 's01e05c01t', 's01e05c02t', 's01e05c03t', 's01e05c04t', 's01e05c05t', 's01e05c06t', 's01e05c07t', 's01e07c01t', 's01e07c02t', 's01e07c03t', 's01e07c04t', 's01e07c05t', 's01e09c00t', 's01e09c01t', 's01e09c02t', 's01e09c03t', 's01e09c04t', 's01e09c05t', 's01e09c06t', 's01e07c00t', 's01e09c07t', 's01e10c00t', 's01e10c01t', 's01e10c02t', 's01e10c03t', 's01e10c05t', 's01e10c07t', 's01e11c00t', 's01e11c01t', 's01e11c02t', 's01e11c03t', 's01e11c04t', 's01e11c05t', 's01e11c06t', 's01e11c07t', 's01e11c0

In [56]:
speaker_set = set()
count = 0
for split in ["train", "dev", "test"]:
    # print(split)
    for scene_id in split_dict[split]:
        # if scene_id != "s01e01c00t":
        #     continue
        # print(scene_id, scene_id[9])
        if scene_id[9]=="f":
            for token in data[scene_id]['speakers']:
                speaker_set.add(token)
            # print(scene_id)
            # print(data[scene_id]['speakers'])
            count += 1
print(count)
print(len(speaker_set))

243
104


In [53]:
print(speaker_set)
print(len(speaker_set))

{'Howard ’s Mother', 'Random woman', 'Korean', 'Cole', 'Girls', 'Waitress', 'Leonard and Howard', 'Girl', 'Professor Proton', 'Man in seat behind', 'Sheldon', 'Random voice', 'Female string quartettist', 'Woman', 'Phone', 'Kathy', 'Dave', 'Barry', 'Dr Koothrappali', 'Photographer', 'Soup Kitchen Manager', 'Various others', 'All', 'Penny', 'Security Guard', 'Taxi Driver', 'Headmaster', 'Raj and Penny together', 'Dr Gablehouser', 'Mr Rostenkowski', 'Doug', 'Lakshmi', 'Leslie Winkle', 'Class instructor', 'Howard and Bernadette', 'Adam', 'Tech Support Voice', 'Mrs Koothrappali', 'Doctor', 'Dimitri', 'Fat Raj', 'Penny ’s Dad', 'Lady Vet', 'Wil', 'Mr. Rostenkowski', 'Zombie', '4 . Sheldon', 'Gablehauser', 'Janine', '3 .   Sheldon', 'Elizabeth', 'Woman in lab coat', 'Raj', '2 .   Amy', 'Elon', 'Priya', 'Wil Wheaton', 'Mrs Koothrapalli', 'Charlie Sheen', 'Bespectacled Man', 'Good looking man coming up stairs', 'Receptionist', 'Assistant', 'Man in Penny ’s Apartment', 'Kripke', 'Howard ’s phone

In [29]:
temp = []
for split in ['train', 'dev', 'test']:
    with open('data/experiment_inputs/dialogue_english/'+split+".english.512.jsonlines", 'r') as f:
        reader = jsonlines.Reader(f)
        for line in reader:
            temp.append(line)

In [54]:
speaker_set = set()
for instance in temp:
    if instance['doc_key'][9]=="t":
        continue
    # print(instance['doc_key'])
    # print(instance['speakers'])
    # for speaker in instance['speakers']:
    #     speaker_set.add(speaker.lower())
    for item in instance['speakers']:
        for token in item[1:-1]:
            if token !="#":
                speaker_set.add(token)

In [46]:
print(speaker_set)
print(len(speaker_set))

{'Korean', 'Cole', 'Girls', 'Waitress', 'Girl', 'Sheldon', 'Montage', 'Enormous', 'Woman', 'Professor', 'Phone', 'Kathy', 'Dave', 'Barry', 'Photographer', 'All', 'Penny', 'Baggage', 'Headmaster', 'Doug', 'Lakshmi', 'Adam', 'Doctor', 'Dimitri', 'Third', 'Wil', 'Zombie', 'Gablehauser', 'Janine', 'Lady', 'Elizabeth', 'Raj', 'Store', 'Female', 'Elon', 'Priya', 'Nerdy', 'Charlie', 'Receptionist', 'Assistant', 'Kripke', 'Random', 'Everybody', 'Spock', 'Gretchen', 'Angela', 'Kevn', 'Santa', 'Bermadette', 'Soup', 'Class', 'Ramona', 'Good', 'Strange', 'Summer', 'Audiitoner', '3', 'Answerphone', 'Emily', 'Mr', 'Director', 'Lesley', 'Kevin', 'Christie', 'Bespectacled', 'Cooper', 'Siri', 'Security', 'Fat', 'Emly', 'Josh', 'Howard', 'Witch', '5', 'Shelldon', 'Pizza', 'Dr', 'Waiter', 'Stuart', 'Trent', 'Amy', 'Lucy', 'Elderly', 'Gablehouser', 'Howie', 'Tech', 'Ms', 'Katee', 'Todd', 'Arthur', 'Missy', 'Nurse', 'Dead', 'Toby', 'First', 'Official', 'Rajj', 'Vet', 'Mr.', 'Zack', 'Nathan', 'Alex', 'Mike'

In [35]:
a = [['[SPL]', 'Ross', 'Ross', 'Ross', '#', '#', '#', '#', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Monica', 'Monica', 'Monica', 'Monica', 'Monica', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Joey', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Ross', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', '[SPL]']]

for item in a:
    print(item)

['[SPL]', 'Ross', 'Ross', 'Ross', '#', '#', '#', '#', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', 'Phoebe', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', '#', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Rachel', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Monica', 'Monica', 'Monica', 'Monica', 'Monica', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', 'Chandler', '